In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!which python

/net/bs-filesvr01/export/group/hierlemann/spikesorting/miniconda3/envs/si2/bin/python


In [3]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents as sc
import belextractors as be
import spikeinterface.widgets as sw
import spikeinterface.extractors as se

from pathlib import Path
import matplotlib.pyplot as plt

import h5py
import numpy as np
import spikeinterface.toolkit as tk
import time

%matplotlib widget

In [4]:
#Config p1c2 (HD)
ncmos_path = Path("/net/bs-filesvr02/export/group/hierlemann/intermediate_data/NeuroCMOS/aforastiere/Thesis/First neuroplating/20220614-Hasan/Chip148/ncmos_20220614_141348.h5")

In [5]:
rec = be.NeuroCMOSRecordingExtractor(ncmos_path, 'rec00000')

In [ ]:
fig=plt.figure()
si.plot_probe_map(rec)
plt.savefig("Electrode map configuration.pdf")
#plt.colorbar(label="prova")

In [6]:
rec_f = si.bandpass_filter(rec)

In [ ]:
peaks = sc.detect_peaks(rec_f, n_jobs=16, total_memory="1G", progress_bar=True)

In [ ]:
len(peaks)

In [ ]:
un_ind, counts = np.unique(peaks["channel_ind"], return_counts=True)

In [ ]:
rates = counts / (rec.get_num_samples() / rec.get_sampling_frequency())

In [ ]:
plt.figure()
_ = plt.hist(rates,bins=100)
plt.ylabel('Number of electrodes')
plt.xlabel('Spikes frequency (Hz)')
plt.savefig("Electrode activity.pdf")


In [ ]:
electrodes = rec_f.get_property("electrode")

In [ ]:
electrode_rates = {}
for (ind, rate) in zip(un_ind, rates):
    electrode_rates[electrodes[ind]] = rate

In [ ]:
#accoppia elettrodo con il rate rilevato prima dello spike sorting
electrode_rates

In [ ]:
si.plot_peak_activity_map(rec_f, peaks)
fig = plt.gcf()
ax = plt.gca()
fig.colorbar(ax.get_children()[-2],label='Frequency (Hz)')
plt.title('')

In [ ]:
import pickle
pickle.dump(fig, open('Electrodemapactivity.pkl', 'wb')) 
figx = pickle.load(open('Electrodemapactivity.pkl', 'rb'))

In [ ]:
si.plot_probe_map(rec,show_channel_on_click = True)

In [ ]:
rec_f.get_channel_ids()[1652]

In [ ]:
sort_tdc = si.run_tridesclous(rec_f, output_folder="scan", n_jobs_bin=8, total_memory="2G",
                              verbose=True)

In [7]:
sort_tdc = si.TridesclousSortingExtractor("/links/groups/hierlemann/Temp/Alessandro/SpikeSorting/Hasan's chips/20220614-Hasan/Chip148/141348/scan")

h5py version > 2.10.0. Some extractors might not work properly. It is recommended to downgrade to version 2.10.0: 
>>> pip install h5py==2.10.0


TypeError: 'NoneType' object is not subscriptable

In [ ]:
sort_tdc = si.TridesclousSortingExtractor("scan/")

In [ ]:
si.plot_rasters(sort_tdc)
plt.ylabel('Neuron')
plt.xlabel('Time (s)')
fig=plt.gcf()
ax=plt.gca()

In [ ]:
pickle.dump(fig, open('Raster.pkl', 'wb')) 
figx = pickle.load(open('Raster.pkl', 'rb'))

In [ ]:
we = si.extract_waveforms(rec_f, sort_tdc, "retina_wf_tdc",ms_before=1.5,
    ms_after=2.,    max_spikes_per_unit=500, n_jobs=8,
                          total_memory="2G", progress_bar=True, load_if_exists=True, 
                          return_scaled=True)

In [ ]:
templates = we.get_all_templates()

In [ ]:
len(templates)

In [ ]:
fig, ax = plt.subplots()
axes = [ax] * len(templates)
w = si.plot_unit_templates(we, radius_um=50, axes=axes)
plt.title('')

In [ ]:
pickle.dump(fig, open('Templates.pkl', 'wb')) 
figx = pickle.load(open('Templates.pkl', 'rb'))

In [ ]:
unit_ids = sort_tdc.unit_ids[:]
si.plot_unit_templates(we, unit_ids=[0])

In [ ]:
sw.plot_unit_waveforms(we, unit_ids=[0])

In [ ]:
si.plot_crosscorrelograms(sort_tdc, unit_ids=[3,10], symmetrize=True)
fig=plt.gcf()
ax=plt.gca()
names=["Counts","Counts","Counts","Counts"]
for i,name in enumerate(names) :
    plt.subplot(2,2,i+1)
    plt.ylabel(name)
    plt.xlabel('Time delay (ms)')
    

In [ ]:
pickle.dump(fig, open('CGs.pkl', 'wb')) 
figx = pickle.load(open('CGs.pkl', 'rb'))

In [ ]:
#si.plot_unit_probe_map(we, unit_ids=[23])

In [ ]:
#si.plot_crosscorrelograms?

In [ ]:
#si.plot_peak_activity_map?

In [ ]:
w_isi = sw.plot_isi_distribution(sort_tdc, window_ms=150.0, bin_ms=5.0)

In [ ]:
unit_ids = sort_tdc.unit_ids[:]
# sw.plot_unit_waveform_density_map(we, unit_ids=unit_ids, max_channels=5)

In [ ]:
# sw.plot_amplitudes_distribution(we)
# plt.ylabel('Amplitude?')
# plt.xlabel('Neuron')

In [ ]:
# sw.plot_amplitudes_timeseries(we)

In [ ]:
pickle.dump(fig1, open('Activ', 'wb')) 
figx = pickle.load(open('Activ', 'rb'))

In [ ]:
unit_ids

In [ ]:
#firing rate
freq=[]
for un in unit_ids:
    st = sort_tdc.get_unit_spike_train(unit_id=un, segment_index=0)
    print('Num. events for unit 1seg0 = {}'.format(len(st)/123.5))
    freq.append(len(st)/123.5)


In [ ]:
fig, ax = plt.subplots()
plt.stem(unit_ids,freq,'o')
plt.xlabel('Neuron')
plt.xticks(unit_ids)
plt.ylabel('Firing rate (Hz)')
plt.savefig("Neuron firing rate.pdf")

In [ ]:
#unit waveforms
unit_ids = sort_tdc.unit_ids

for unit_id in unit_ids:
    wfs = we.get_waveforms(unit_id)
    print(unit_id, ':', wfs.shape)

In [ ]:
colors = ['Olive', 'Teal', 'Fuchsia']

fig, ax = plt.subplots()
for i, unit_id in enumerate(sort_tdc.unit_ids[1:3]):
    wf = we.get_waveforms(unit_id)
    color = colors[i]
    ax.plot(wf[:, :, 1196].T, color=color, lw=0.3) # [x,y,z] plot x waveforms on channel z=1196 on y seconds

In [ ]:
#plot the templates
for unit_id in unit_ids[:3]:
    fig, ax = plt.subplots()
    template = we.get_template(unit_id=unit_id, mode='average')
    print(template.shape) #(num seconds, num wf)
    ax.plot(template*LSB*1e6)
    ax.set_ylabel('Amplitude(\u03BCV)')
    ax.set_xlabel('Number of samples')
    ax.set_title(f'{unit_id}')

plt.show()

In [ ]:
fig, ax = plt.subplots()

wf = we.get_waveforms(0)

ax.plot(wf[:, :, 1196 ].T, color=color, lw=0.1) # [x,y,z] plot x waveforms on channel z=8 on y number of samples

In [ ]:
#search the channels for each unit having the highest peak
extremum_channels_ids = tk.get_template_extremum_channel(we, peak_sign='neg')
print(extremum_channels_ids)

In [ ]:
myvar = extremum_channels_ids
  
# Open a file and use dump()
with open('AISpos.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(myvar, file)

In [ ]:
type(template)

In [ ]:
firing_rates = si.compute_firing_rate(we)
print(firing_rates)
isi_violation_ratio, isi_violations_rate, isi_violations_count = si.compute_isi_violations(we)
print(isi_violation_ratio)
snrs = si.compute_snrs(we)
print(snrs)

In [ ]:
si.compute_isi_violations?

In [ ]:
si.compute_snrs?

In [ ]:
w_ach = sw.plot_autocorrelograms(sort_tdc, window_ms=100.0, bin_ms=1.0, unit_ids=[3,10], symmetrize=True)


In [ ]:
we.get_template?

In [ ]:
we.get_waveforms?

In [ ]:
si.plot_crosscorrelograms?

In [ ]:
np.shape(wf)

In [ ]:
np.shape(template)

In [ ]:
sig = rec.get_traces(channel_ids= [b'vrbN595'],return_scaled=False)

t = np.linspace(0, len(sig)/20000,len(sig))
LSB=(2/2024)/(1023);
fs =20e3;
sig2 = sig*LSB*1e6
sig2 = sig2-np.mean(sig2)

plt.figure()
plt.plot(t,sig2)
plt.show()
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (\u03BCV)')

In [ ]:
t = np.linspace(0, len(sig)/20000,len(sig))
LSB=(2/2024)/(1023);
fs =20e3;
sig2 = sig*LSB*1e6
sig2 = sig2-np.mean(sig2)

plt.figure()
plt.plot(t,sig2)
plt.show()
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (\u03BCV)')